# Emerging Technologies - Problems

This notebook contains solutions to the problems assigned for the Emerging Technologies module.

**Author:** Tommy Mitchell  
**Module:** Emerging Technologies  
**Institution:** ATU

## Problem 1: Generating Random Boolean Functions

The **Deutsch–Jozsa algorithm** is designed to work with functions that accept a fixed number of Boolean inputs and return a single Boolean output. Each function is guaranteed to be either:

- **Constant**: Always returns `False` or always returns `True` for all inputs
- **Balanced**: Returns `True` for exactly half of the possible input combinations

Question 1 is to write a Python function `random_constant_balanced` that returns a randomly chosen function from the set of constant or balanced functions taking four Boolean arguments as inputs.

---

### Background and Research

The Deutsch-Jozsa algorithm, developed by David Deutsch and Richard Jozsa in 1992[1], represents one of the first demonstrations of quantum computational advantage. The promblem is esoteric being more of a proof of concept, but it provides a clear example where a quantum algorithm exponentially outperforms any deterministic classical algorithm.

**Classical vs Quantum Complexity:**
- **Classical (deterministic)**: In the worst case, we need to query $2^{n-1} + 1$ inputs to determine if an n-bit function is constant or balanced
- **Quantum**: Only **1 query** is needed, regardless of the number of input bits

For a 4-bit input function worst case:
- Classical: 9 queries
- Quantum: 1 query

**References:**
1. Deutsch, D., & Jozsa, R. (1992). "Rapid solution of problems by quantum computation." *Proceedings of the Royal Society of London A*, 439(1907), 553-558.
2. Nielsen, M. A., & Chuang, I. L. (2010). *Quantum Computation and Quantum Information*. Cambridge University Press.
3. IBM Qiskit Textbook: [Deutsch-Jozsa Algorithm](https://learning.quantum.ibm.com/course/fundamentals-of-quantum-algorithms/quantum-query-algorithms)

### Understanding Constant and Balanced Functions

For a function $f: \{0,1\}^n \rightarrow \{0,1\}$ with 4 Boolean inputs ($n=4$), there are $2^4 = 16$ possible input combinations.

**Constant functions** (2 total):
- $f(x) = 0$ for all inputs (always False)
- $f(x) = 1$ for all inputs (always True)

**Balanced functions**: Return True for exactly 8 of the 16 inputs, and False for the other 8. The number of such functions is $\binom{16}{8} = 12,870$.

Our `random_constant_balanced` function will randomly select either a constant or balanced function and return it as a callable Python function.

### Implementation: `random_constant_balanced` Function

The implementation strategy:
1. Randomly choose to generate a constant or balanced function
2. If constant: randomly choose True or False as the return value
3. If balanced: randomly select exactly 8 of the 16 possible inputs to return True

We use closures to capture the function's behavior, allowing the returned function to remember which inputs map to True.

In [7]:
import random
from itertools import product
from typing import Callable


def random_constant_balanced() -> Callable[[bool, bool, bool, bool], bool]:
    """
    Generate a random function that is either constant or balanced.
    
    The function takes four Boolean arguments and returns a single Boolean.
    
    Constant functions:
        - Always return True, OR
        - Always return False
    
    Balanced functions:
        - Return True for exactly half (8) of the 16 possible inputs
        - Return False for the other half
    
    Returns:
        A callable function f(b1, b2, b3, b4) -> bool that is either
        constant or balanced.
    
    Example:
        >>> f = random_constant_balanced()
        >>> f(True, False, True, False)  # Returns True or False
    """
    # Generate all 16 possible input combinations for 4 Boolean arguments
    all_inputs = list(product([False, True], repeat=4))
    
    # Randomly decide: constant (True) or balanced (False)
    is_constant = random.choice([True, False])
    
    if is_constant:
        # Constant function: always returns the same value
        constant_value = random.choice([True, False])
        
        def constant_function(b1: bool, b2: bool, b3: bool, b4: bool) -> bool:
            """A constant function that always returns the same value."""
            return constant_value
        
        # Store metadata for testing/verification
        constant_function.function_type = "constant"
        constant_function.constant_value = constant_value
        return constant_function
    
    else:
        # Balanced function: returns True for exactly half the inputs
        # Randomly select 8 inputs that will return True
        true_inputs = set(random.sample(all_inputs, 8))
        
        def balanced_function(b1: bool, b2: bool, b3: bool, b4: bool) -> bool:
            """A balanced function that returns True for exactly half the inputs."""
            return (b1, b2, b3, b4) in true_inputs
        
        # Store metadata for testing/verification
        balanced_function.function_type = "balanced"
        balanced_function.true_inputs = true_inputs
        return balanced_function

### Demonstration and Verification

We can test our `random_constant_balanced` function by:
1. Generating several random functions
2. Evaluating them on all 16 possible inputs
3. Verifying if they are constant or balanced

In [8]:
def verify_function(f: Callable[[bool, bool, bool, bool], bool]) -> str:
    """
    Verify whether a function is constant or balanced by evaluating all inputs.
    
    Args:
        f: A function taking four Boolean arguments
        
    Returns:
        A string describing the function type and its outputs
    """
    all_inputs = list(product([False, True], repeat=4))
    outputs = [f(*inp) for inp in all_inputs]
    
    true_count = sum(outputs)
    false_count = len(outputs) - true_count
    
    if true_count == 0:
        return f"CONSTANT (always False) - True: {true_count}, False: {false_count}"
    elif true_count == 16:
        return f"CONSTANT (always True) - True: {true_count}, False: {false_count}"
    elif true_count == 8:
        return f"BALANCED - True: {true_count}, False: {false_count}"
    else:
        return f"INVALID - True: {true_count}, False: {false_count}"


# Generate and test 10 random functions
print("Testing random_constant_balanced() function:\n")
print("-" * 60)

for i in range(10):
    f = random_constant_balanced()
    result = verify_function(f)
    func_type = getattr(f, 'function_type', 'unknown')
    print(f"Function {i+1}: {result}")
    print(f"           Metadata type: {func_type}")
    print("-" * 60)

Testing random_constant_balanced() function:

------------------------------------------------------------
Function 1: BALANCED - True: 8, False: 8
           Metadata type: balanced
------------------------------------------------------------
Function 2: BALANCED - True: 8, False: 8
           Metadata type: balanced
------------------------------------------------------------
Function 3: CONSTANT (always False) - True: 0, False: 16
           Metadata type: constant
------------------------------------------------------------
Function 4: CONSTANT (always False) - True: 0, False: 16
           Metadata type: constant
------------------------------------------------------------
Function 5: BALANCED - True: 8, False: 8
           Metadata type: balanced
------------------------------------------------------------
Function 6: CONSTANT (always True) - True: 16, False: 0
           Metadata type: constant
------------------------------------------------------------
Function 7: BALANCED - T